In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.neighbors import KNeighborsClassifier
import math

## 3.1
### 1. We load all the images into training and test sets and resize them to 128x128:

In [3]:
image_train = []
image_test = []
label_train = []
images = []
directory = "./dataset"

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    path = directory + "/" + filename
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (128, 128))
    if "train" in filename:
        label_train.append(filename.split("_")[0])
        image_train.append(img)
    else:
        image_test.append(img)
    images.append(img)
    
image_train = np.array(image_train)
image_test = np.array(image_test)
label_train = np.array(label_train)

In [ ]:
max_col = 4
nbins = 8
def plot_all_hog(gradient):
    nb_row = math.ceil(nbins / max_col) * 100
    nb_plot = nb_row + max_col * 10
    plt.figure(figsize = (20,10))
    for i in range(nbins):
        plt.subplot(nb_plot + i + 1)
        plt.pcolor(gradient[:, :, i])
        plt.gca().invert_yaxis()
        plt.gca().set_aspect('equal', adjustable='box')
        plt.title("HOG bin = " + str(i)), plt.xticks([]), plt.yticks([])
        plt.colorbar()
    plt.show()

### 2. HoG Features:

In [ ]:
def hog(img, plot=True):
    cell_size = (4, 4)  # h x w in pixels
    block_size = (4, 4)  # h x w in cells
    nbins = 8  # number of orientation bins

    hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                      img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

    n_cells = (img.shape[0] // cell_size[0], img.shape[1] // cell_size[1])

    # Compute HoG features
    hog_feats = hog.compute(img)\
                   .reshape(n_cells[1] - block_size[1] + 1,
                            n_cells[0] - block_size[0] + 1,
                            block_size[0], block_size[1], nbins) \
                   .transpose((1, 0, 2, 3, 4))  # index blocks by rows first


    # hog_feats now contains the gradient amplitudes for each direction,for each cell of its group for each group.
    # Indexing is by rows then columns.

    # computation for BlockNorm
    gradients = np.full((n_cells[0], n_cells[1], nbins), 0, dtype=float)
    cell_count = np.full((n_cells[0], n_cells[1], 1), 0, dtype=int)

    for off_y in range(block_size[0]):
        for off_x in range(block_size[1]):
            gradients[off_y:n_cells[0] - block_size[0] + off_y + 1,
                      off_x:n_cells[1] - block_size[1] + off_x + 1] += \
                hog_feats[:, :, off_y, off_x, :]
            cell_count[off_y:n_cells[0] - block_size[0] + off_y + 1,
                       off_x:n_cells[1] - block_size[1] + off_x + 1] += 1

    # Average gradients
    gradients /= cell_count
    
    # Preview
    if plot:
        plt.figure(figsize = (10,10))
        plt.subplot(121)
        plt.imshow(img, cmap='gray')
        plt.title("Original Image"), plt.xticks([]), plt.yticks([])
        plt.show()
        bin = 0 # angle is 360 / nbins * direction
        plot_all_hog(gradients)
        plt.show()
    return gradients

def hog_images(images, plot=False):
    imgs = []
    for img in images:
        val = hog(img, plot)
        imgs.append(val)
    
    return imgs

hog_train_imgs = np.array(hog_images(image_train))

In [ ]:
from random import shuffle


neigh = KNeighborsClassifier()
train_vals = hog_train_imgs.reshape(hog_train_imgs.shape[0], -1)
neigh.fit(train_vals, label_train)

In [ ]:
test_vals = np.array(hog_images(image_test, plot=True))
test_vals = test_vals.reshape(test_vals.shape[0], -1)

print(neigh.predict(test_vals))

## 3.2 Reasoning Questions
### 1)

HoG is not rotation invariant meaning that if you calculate HoG on an image and calculate HoG on a rotated version of that image, the results will not be the same. For a classification example you could also include rotated images into your training set to make your classifier better recognize them. Another approach could be to use another feature detector such as SIFT which is rotation invariant. One approach to ensure all logos have a uniform orientation is to maintain one image for each class we are trying to classify that we know is oriented in the right direction. Then for every logo that comes in, we would need to calculate the SIFT features, match them between the two images and rotate the images such that they match. 

#### 2)
There would be two gradients needed to accurately depict the picture. This is because with only one gradient, you would not be able to distinguish between the left most image and the other two images. With the second gradient direction we would be able to have two reference points and uniquely identify all the images. 